# K-nearest neigbours

##### Imports

In [1]:
import time
import numpy as np
import os
from scipy.io import wavfile

# sklearn
from sklearn import neighbors
from sklearn.metrics import confusion_matrix

# visualization
import matplotlib.pyplot as plt
import seaborn

# torch
import torch
from torchvision import datasets, transforms

# Dataset
from torch.utils.data import DataLoader, Dataset

#Sklearn
from sklearn.model_selection import train_test_split

from audio_dataset import AudioDataset as AD


##### AudioDataset class

In [9]:
sliced_dataset = "../data/short_audio_dataset"
sliced_dataset_lenght = 16050
original_dataset = "../data/audio_dataset"
original_dataset_lenght = 80249

class AudioDataset(Dataset):
    def __init__(self, drop_both=False, use_short=False):
        root_folder = original_dataset if not use_short else sliced_dataset
        max_length = original_dataset_lenght if not use_short else sliced_dataset_lenght
        self.class_map = {"both": 0, "esben" : 1, "peter": 2}
        self.data = []
        self.labels = []
        for subdir, dirs, files in os.walk(root_folder):
            for file_name in files:
                if "both" in subdir and drop_both:
                   continue
                file_path = os.path.join(subdir, file_name)
                _, wav = wavfile.read(file_path)
                if wav.shape[0] > max_length:
                    max_length = wav.shape[0]
                    print("Found wav with more length than specified max one, new max is:", wav.shape[0])
                wav = np.pad(wav, (0, max_length-wav.shape[0]))
                label = file_path.split('/')[3][2:]
                self.labels.append(label)
                self.data.append(wav)
        print("Max length of wav files:", max_length)
    

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        wav = self.data[idx]
        label = self.labels[idx]
        class_id = self.class_map[label]
        wav_tensor = torch.from_numpy(wav)
        class_id = torch.tensor([class_id])
        return wav_tensor, class_id


##### Initialize the Dataset

In [2]:
dataset = AD(drop_both=True, use_short=False, normalize=True)
data_len = len(dataset)
train_size, test_size, valid_size = int(data_len * 0.8), int(data_len * 0.1), int(data_len * 0.1)

data_train, data_test, data_valid = torch.utils.data.random_split(dataset, (train_size, test_size, valid_size))

Start reading files
Loaded DATABASE from ../data/audio_dataset
200 total file
Longest file is 80249 long
Mean: -0.6988648176193237
Standard deviation: 2332.404541015625
Normalization: True


##### Define knn algorithm

In [15]:
def knn_param_search(train_data, train_labels, test_data, test_labels, 
                     metrics=('manhattan', 'euclidean', 'chebyshev'), 
                     ks=(1, 3, 5, 10, 25, 50, 100, 250), 
                     n_train=None, n_test=None, algorithm='brute'):
  """
  Takes a dataset and plots knn classification accuracy 
  for different hyper parameters.

  n_train and n_test allows to subsample the dataset for faster iteration
  """
  x_train = np.array(train_data)
  y_train = np.array(train_labels)
  x_test = np.array(test_data)
  y_test = np.array(test_labels)
  
  # subsample the dataset
  if n_train:
    x_train, y_train = x_train[:n_train], y_train[:n_train]
  if n_test:
    x_test, y_test = x_test[:n_test], y_test[:n_test]

  for metric in metrics:
    print(f'Metric: {metric}')
    for k in ks:
        print(f'\tk: {k:3d} Training', end='')
        classifier = neighbors.KNeighborsClassifier(k, algorithm=algorithm, metric=metric)
        classifier = classifier.fit(x_train, y_train)

        start = time.time()
        print(f'\r\tk: {k:3d} Testing', end='')
        labels = classifier.predict(x_test)
        duration = time.time() - start

        correct = labels == np.array(y_test)
        print(f'\r\tk: {k:3d} Accuracy: {correct.mean() * 100:.2f} %, Duration: {duration:.2f} s')

##### Call the KNN Algorithm 

In [20]:
train_data = []
train_labels = []
valid_data = []
valid_labels = []
for x,y in data_train:
    train_data.append(x)
    train_labels.append(y)
for x,y in data_valid:
    valid_data.append(x)
    valid_labels.append(y)


In [18]:
knn_param_search(train_data, 
                 train_labels, 
                 valid_data, 
                 valid_labels,
                 ks=[3]
                )

/tmp/ipykernel_42433/2294672208.py:11: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  x_train = np.array(train_data)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (160,) + inhomogeneous part.